# Agent Testing Notebook

This notebook is for testing the `LangGraphAgent` with different agent architectures.

## Setup

In [1]:
from langchain_community.tools.tavily_search import TavilySearchResults
import os
from dotenv import load_dotenv
load_dotenv()
from typing import Any, Callable, List, Optional, cast, Dict, Literal, Union
from pydantic import BaseModel, Field, field_validator
from langchain.tools import BaseTool, tool
from langchain_core.language_models import BaseChatModel

In [2]:
# from src.langgraph.app.core.langgraph.smolagent.smol_agent import SMOLAgent
from src.langgraph.app.core.langgraph.agents import create_agent

In [3]:

import os
from typing import Literal
from langchain_community.tools.tavily_search import TavilySearchResults
import os
from dotenv import load_dotenv
load_dotenv()
from typing import Any, Callable, List, Optional, cast, Dict, Literal, Union
from pydantic import BaseModel, Field, field_validator
from langchain.tools import BaseTool, Tool


class SearchToolInput(BaseModel):
    query: str = Field(..., description="The search query to look up.")
    max_results: Optional[int] = Field(default=10, description="The maximum number of search results to return.")

# Define the Tool
class TavilySearchTool:
    def __init__(self, max_results: int = 10):
        self.max_results = max_results

    def search(self, query: str) -> Optional[List[Dict[str, Any]]]:
        """
        Perform a web search using the Tavily search engine.
        """
        try:
            # Initialize the Tavily search tool with the configured max_results
            search_tool = TavilySearchResults(max_results=self.max_results, tavily_api_key=os.getenv("TAVILY_API_KEY"))

            # Perform the search (synchronously)
            result = search_tool.invoke({"query": query})

            # Return the search results
            return result
        except Exception as e:
            return {"error": str(e)}

tavily_search_tool = Tool(
    name="Tavily_Search",
    func=TavilySearchTool().search,
    description="Performs web searches using the Tavily search engine, providing accurate and trusted results for general queries.",
    args_schema=SearchToolInput
)


In [4]:
tool_2 = Tool(
    name="Echo_Tool",
    func=lambda x: f"Echo: {x}",
    description="A tool that echoes the input it receives.",
    args_schema=SearchToolInput
)

tool_3 = Tool(
    name="Echo_Tool_2",
    func=lambda x: f"Echo_2: {x}",
    description="A tool that echoes the input it receives.",
    args_schema=SearchToolInput
)

## 1. Base Agent Test

In [5]:
from src.langgraph.app.core.langgraph.toolsagent.agents import create_agent
from langgraph.checkpoint.memory import MemorySaver
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4", temperature=0)
selector_llm = ChatOpenAI(model="gpt-4", temperature=0)

memory = MemorySaver()

In [6]:

prompt = """You are an intelligent agent that can perform web searches to gather information."""

tools = [tool_2, tool_3, tavily_search_tool]
# Create a dictionary from the list, using the tool's name as the key
tool_registry_dict = {tool.name: tool for tool in tools}

agent = create_agent(
    selector_llm=selector_llm,
    main_llm=llm,
    tool_registry=tool_registry_dict,
    tool_selection_limit=1,
    prompt=prompt,
    checkpointer=memory,
    # config=None
)

In [11]:
from langchain_core.messages import HumanMessage
result = await agent.ainvoke({
    "messages": [HumanMessage(content="what is your name?")],
    "remaining_steps": 15,
}, config={"configurable": {"thread_id": "thread_id"}})


In [12]:
result

{'messages': [HumanMessage(content='What is the capital of France?', additional_kwargs={}, response_metadata={}, id='88da06d7-7806-4f58-b1d4-5ae22aff0294'),
  AIMessage(content="Selected tools: ['Tavily_Search']. Reasoning: The user's query is a general knowledge question that can be answered by performing a web search. Therefore, the Tavily_Search tool, which performs web searches, is the most relevant tool for this query.", additional_kwargs={}, response_metadata={}, id='a4500f78-e9d7-417f-8744-87f08e3bdd69'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_YYcwhYe3dNqULoQ94c2V18ez', 'function': {'arguments': '{\n  "query": "capital of France"\n}', 'name': 'Tavily_Search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 171, 'total_tokens': 190, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_to

In [2]:
import asyncio
import nest_asyncio
nest_asyncio.apply()
import os
import logging
from typing import List, Sequence, TypedDict, Annotated, Literal, Optional, Dict, Any
from dotenv import load_dotenv

from langchain_core.messages import BaseMessage, HumanMessage
from src.langgraph.app.core.langgraph.agents import create_agent
from langchain_openai import ChatOpenAI
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.graph.state import CompiledStateGraph
from langgraph.graph.message import add_messages
from langgraph.checkpoint.base import BaseCheckpointSaver
from langgraph.managed import RemainingSteps

# Import local base tools and the system prompt

SMOL_AGENT_PROMPT = """You are a highly intelligent and autonomous agent designed to assist users with a variety of tasks using a predefined set of local tools. You do not have access to external MCP servers or APIs. Your goal is to provide accurate and helpful responses based on the user's input and the capabilities of your tools.
"""

# Load environment variables from .env file
load_dotenv()

# --- Production-Ready Logging ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)


# --- Enhanced Agent Configuration & State ---
class AgentState(TypedDict):
    """
    Defines the state of the agent. This is the central data structure that flows
    through the graph. Using LangGraph's `RemainingSteps` provides robust,
    built-in loop protection.

    Attributes:
        messages: The history of messages in the conversation.
        remaining_steps: The number of steps left before execution is halted.
    """
    messages: Annotated[Sequence[BaseMessage], add_messages]
    remaining_steps: RemainingSteps

class AgentConfig(TypedDict, total=False):
    """
    A schema for configuring the agent's compiled graph, allowing for
    interrupts before or after specific nodes.
    """
    interrupt_before: List[str]
    interrupt_after: List[str]


class SMOLAgent:
    """
    A robust, simplified autonomous agent that uses a predefined set of local,
    base tools. It does not connect to any external MCP servers.
    """

    def __init__(self, model_name: str = "gpt-4o", max_steps: int = 15, checkpointer: Optional[BaseCheckpointSaver] = None, tools = None):
        """
        Initializes the agent's configuration.

        Args:
            model_name: The specific OpenAI model name to use (e.g., "gpt-4o").
            max_steps: The maximum number of LLM calls before forcing a stop.
            checkpointer: An optional LangGraph checkpointer for state persistence and memory.
        """
        self.model_name = model_name
        self.max_steps = max_steps
        self.checkpointer = checkpointer
        self.tools: list = tools if tools is not None else base_tools
        self.executor: Optional[CompiledStateGraph] = None # Compiled agent graph executor

    async def _build_executor(self, config: Optional[AgentConfig] = None):
        """
        Builds and compiles the agent graph using LangChain's create_agent factory.
        This method is called lazily to ensure tools are loaded before compilation.
        """
        if self.executor:
            return

        logger.info("Building and compiling the sports agent executor...")
        
        # Instantiate the language model
        llm = ChatOpenAI(model=self.model_name, temperature=0, streaming=True)
        
        # Use LangChain's factory to create the standard ReAct agent graph
        self.executor = create_agent(
            model=llm,
            tools=self.tools,
            prompt=SMOL_AGENT_PROMPT,
            state_schema=AgentState,
            checkpointer=self.checkpointer,
            interrupt_before=config.get("interrupt_before") if config else None,
            interrupt_after=config.get("interrupt_after") if config else None
        )
        logger.info("Sports agent executor compiled successfully.")


    async def ainvoke(self, messages: Sequence[BaseMessage], thread_id: str) -> Dict[str, Any]:
        """
        Asynchronously invokes the agent to get the final result in a single call.
        This is ideal for multi-agent systems where one agent's complete output is
        the input for another.

        Args:
            messages: The list of BaseMessage objects representing the conversation history.
            thread_id: A unique identifier for the conversation thread for memory.

        Returns:
            A dictionary representing the final state of the agent's execution.
        """
        await self._build_executor()

        run_config = {"configurable": {"thread_id": thread_id}}
        initial_input = {
            "messages": messages,
            "remaining_steps": self.max_steps,
        }

        logger.info(f"--- Invoking Agent for Thread '{thread_id}' with Messages: {messages} ---")
        
        final_state = await self.executor.ainvoke(initial_input, config=run_config)
        
        logger.info(f"\n--- Final Answer ---\n{final_state['messages'][-1].content}")
        return final_state
    
    
    async def arun(self, messages: Sequence[BaseMessage], thread_id: str, config: Optional[AgentConfig] = None):
        """
        Asynchronously runs the agent with a given list of messages and conversation thread ID.

        Args:
            messages: The list of BaseMessage objects representing the conversation history.
            thread_id: A unique identifier for the conversation thread for memory.
            config: Optional configuration for setting interrupts.
        """
        # Build the executor on the first run
        await self._build_executor(config)

        # Define the per-run configuration, including the thread_id for memory
        run_config = {"configurable": {"thread_id": thread_id}}

        # Prepare the initial input for the agent graph
        initial_input = {
            "messages": messages,
            "remaining_steps": self.max_steps,
        }

        logger.info(f"--- Running Agent for Thread '{thread_id}' with Messages: {messages} ---")
        
        # Stream the agent's execution steps for real-time logging
        try:
            async for chunk in self.executor.astream(initial_input, config=run_config, recursion_limit=150):
                for key, value in chunk.items():
                    if key == "agent" and value.get('messages'):
                        ai_msg = value['messages'][-1]
                        if ai_msg.tool_calls:
                            tool_names = ", ".join([call['name'] for call in ai_msg.tool_calls])
                            logger.info(f"Agent requesting tool(s): {tool_names}")
                        else:
                            logger.info(f"\n--- Final Answer ---\n{ai_msg.content}")

                    elif key == "tools" and value.get('messages'):
                        tool_msg = value['messages'][-1]
                        logger.info(f"Tool executed. Result: {str(tool_msg.content)[:300]}...")
        except Exception as e:
            logger.error(f"An error occurred during agent execution: {e}", exc_info=True)
            





In [20]:
import os
from pathlib import Path

# Get the current working directory (where the notebook/script is running)
cwd = os.getcwd()
print("Current working directory:", cwd)

# Or using pathlib
root_dir = Path.cwd()
print("Root directory (Path):", root_dir)

Current working directory: c:\Users\pault\Documents\3. AI and Machine Learning\2. Deep Learning\1c. App\Projects\morgana\backend
Root directory (Path): c:\Users\pault\Documents\3. AI and Machine Learning\2. Deep Learning\1c. App\Projects\morgana\backend


In [3]:
async def initialize_agent():
    """Main function to instantiate and run the SMOLAgent."""
    from langgraph.checkpoint.memory import MemorySaver

    if not (os.getenv("OPENAI_API_KEY") and os.getenv("TAVILY_API_KEY")):
        raise ValueError("API keys for OpenAI and Tavily must be set in the .env file.")

    memory = MemorySaver()
    agent = SMOLAgent(model_name="gpt-4o", checkpointer=memory, tools=base_tools)
    
    return agent

In [5]:

agent = await initialize_agent()


async def main(agent=None):
    """Main function to instantiate and run the SMOLAgent."""
    if agent is None:
        agent = await initialize_agent()
        
    thread_id = "test_thread_001"
    
    query = "when is the next nba season starting"
    await agent.arun(query, thread_id=thread_id)


if __name__ == "__main__":
    asyncio.run(main())

2025-09-08 13:34:13,987 - __main__ - INFO - Building and compiling the sports agent executor...
2025-09-08 13:34:14,001 - __main__ - INFO - Sports agent executor compiled successfully.
2025-09-08 13:34:14,002 - __main__ - INFO - --- Running Agent for Thread 'test_thread_001' with Messages: when is the next nba season starting ---
2025-09-08 13:34:14,966 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-08 13:34:15,177 - __main__ - INFO - Agent requesting tool(s): tavily_search_tool
2025-09-08 13:34:17,873 - __main__ - INFO - Tool executed. Result: [{"title": "NBA announces schedule for 2023-24 season", "url": "https://www.nba.com/news/2023-24-nba-regular-season-schedule", "content": "The 78th NBA regular season will tip off on Tuesday, Oct. 24, 2023, and conclude on Sunday, April 14, 2024.  The 2024 NBA Play-In Tournament will take place from...
2025-09-08 13:34:18,475 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/c